# Project: Wrangle and Analyze WeRateDogs Twitter Account
## Udactiy Data Analyst Nanodegree Program - Project 8 / Part 1: Data Wrangling
##### Esra Arı
##### 08'18



## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#gathering">Gathering Data</a></li>
<li><a href="#assessing">Assessing Data</a></li>
<li><a href="#cleaning">Cleaning Data</a></li>
<li><a href="#storing">Storing/Exporting Data</a></li>
</ul>

<a id='intro'></a>
## Introduction

This analysis is 8th project of the Udacity Data Analysis Nanodegree program.  Project goal is wrangling WeRateDogs twitter data to create interesting and trustworthy visualizations, both exploratory and predictive analysis trying different machine learning algorithms. 

Due to complexity of project, there are 2 different jupyter notebook project named as wrangle_act and act_report in **bold** at below.

Methodology in this project is given following:
- Data wrangling, which consists of: **(wrangle_act - Data Wrangling)**
  - Gathering data
  - Assessing data 
  -	Cleaning data
  - Storing/Exporting data
  
  
  
- Exploratory Data Analysis **(act_report - Exploratory and Predictive Analyses)**
  - Analyzing data 
  - Visualizing data
  
  
  
- Predictive Data Analysis **(Word documentation which includes all process in summary.)**
 - Editing Metadata
 - Missing Value Treatment
 - Feature Extraction / Feature Hashing
 - Dimension Reduction / Principle Component Analysis
 - Using different sampling techniques such as oversampling 
 - Data splitting
 - Trying different supervised machine learning algorithms with different parameters. (Random forest and boosted decision tree algorithms were applied for this project on Azure network)
 - Reporting on 1) data wrangling efforts and 2) data analyses and visualizations 3) prediction methodology in an executive way with Microsoft word.

<a id='gathering'></a>
## Gathering Data

The three pieces of data gathered for this project as described below.
- The WeRateDogs Twitter archive.
- The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers.
- Each tweet's retweet count and favorite ("like") count at minimum, and any additional data. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API which is mentioned in Twitter API section detailed for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. 

Necessary libraries are dowloaded as following.

In [1]:
import pandas as pd
import numpy as np
import requests as rq
import json as js
import tweepy as tp
import re

Firstly, WeRateDogs twitter arciheve is imported as df1. To gather number of retweet, favoite information belongs to these tweet id's, I am using tweepy API in the following section. It is important to keep in mind that these tweet_id's will be using following section to gather more information. 

In [2]:
df1 = pd.read_csv("twitter-archive-enhanced.csv")
df1.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
85,876120275196170240,NaN,NaN,2017-06-17 16:52:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Venti, a seemingly caffeinated puppoccino...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/876120275...,13,10,Venti,None,None,None,None


The tweet image predictions data imported as df2.

In [3]:
df2 = pd.read_csv("image-predictions.tsv", delimiter='\t',encoding='utf-8')
df2.sample()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
613,680145970311643136,https://pbs.twimg.com/media/CXBdJxLUsAAWql2.jpg,1,miniature_poodle,0.457117,True,toy_poodle,0.226481,True,Maltese_dog,0.067682,True


I created my own account on tweepy API, got following credentials to access API.

In [4]:
consumer_key = 'bL23pGKR1iCxzLupjZhCHXrLO'
consumer_secret = 'llfHsX4dIX4x1j5k0mBVLXRoJfqqoZRFim3mESz10OJhAJEJTR'
access_token = '345309863-rhfxAY4CWg78PmAp5PssNslsU691WbhkqxvpEnb3'
access_secret = '5jRaAuyCOLyg3K77csYTlYgqePEgbfXe6sVGdWR39VIPJ'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tp.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

I created a loop which adds each available tweet to a new line of tweet_json.txt

In [7]:
with open('tweet_json.txt', 'a', encoding='utf8') as f:
    for tweet_id in df1['tweet_id']:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            js.dump(tweet._json, f)
            f.write('\n')
        except:
            continue

Rate limit reached. Sleeping for: 375
Rate limit reached. Sleeping for: 440
Rate limit reached. Sleeping for: 445


Now, I have dowloaded tweet_json.txt file. However, it needs to be mapped and converted to pandas dataframe for further analysis.Following code's loop helps to append each tweet into a list.

In [5]:
tweets_data = []

tweet_file = open('tweet_json.txt', "r")

for line in tweet_file:
    try:
        tweet = js.loads(line)
        tweets_data.append(tweet)
    except:
        continue
        
tweet_file.close()

In [6]:
df3 = pd.DataFrame()

With the help of https://jsoneditoronline.org/ site, json object which belongs to the just one id can be investigated an accoding to that demanded information can be mapped to the list appended into pandas dataframe in the following. 

In [7]:
df3['id'] = list(map(lambda tweet: tweet['id'], tweets_data))
df3['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_data))
df3['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_data))
df3['retweeted'] = list(map(lambda tweet: tweet['retweeted'], tweets_data))
df3['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data))
df3['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
df3['contributors'] = list(map(lambda tweet: tweet['contributors'], tweets_data))
df3['favorited'] = list(map(lambda tweet: tweet['favorited'], tweets_data))
df3['coordinates'] = list(map(lambda tweet: tweet['coordinates'], tweets_data))
df3['place'] = list(map(lambda tweet: tweet['place'], tweets_data))
df3['geo'] = list(map(lambda tweet: tweet['geo'], tweets_data))
df3['place'] = list(map(lambda tweet: tweet['place'], tweets_data))

In [8]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3715 entries, 0 to 3714
Data columns (total 11 columns):
id                3715 non-null int64
retweet_count     3715 non-null int64
favorite_count    3715 non-null int64
retweeted         3715 non-null bool
lang              3715 non-null object
created_at        3715 non-null object
contributors      0 non-null object
favorited         3715 non-null bool
coordinates       0 non-null object
place             1 non-null object
geo               0 non-null object
dtypes: bool(2), int64(3), object(6)
memory usage: 268.5+ KB


<a id='assessing'></a>
## Assessing Data

In this section, each three pandas dataframe gathered previous section will be investigated.  Assesing data will be done both **visiualy**(scrolling through the data in your preferred software application) and **programmatically**(using code to view specific portions and summaries of the data). Both quality and tiddiness issue will be noted end of this section. Also, sources of low quality /dirty and messy/untidy data will be mentioned shortly. 

**Sources of Dirty Data**

***Dirty data = low quality data = content issues***

There are lots of sources of dirty data. Basically, anytime humans are involved, there's going to be dirty data. There are lots of ways in which we touch data we work with.

- user entry errors
- no data coding standards, or having standards poorly applied, causing problems in the resulting data
- integrated data where different schemas have been used for the same type of item
- legacy data systems, where data wasn't coded when disc and memory constraints were much more restrictive than they are now. Over time systems evolve. Needs change, and data changes
- no unique identifiers it should
- lost in transformation from one format to another
- programmer error
- corrupted in transmission or storage by cosmic rays or other physical phenomenon

**Sources of Messy Data**

***Messy data = untidy data = structural issues***

Messy data is usually the result of poor data planning. Or a lack of awareness of the benefits of tidy data. Fortunately, messy data is usually much more easily addressable than most of the sources of dirty data mentioned above.

I wanted to move easy to hard. Therefore, I prefered to start with df2 dataframe which includes image-prediction dataset.

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [10]:
df2.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [11]:
df2.tail()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


In [12]:
df2.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
924,702321140488925184,https://pbs.twimg.com/media/Cb8lWafWEAA2q93.jpg,3,West_Highland_white_terrier,0.769159,True,Scotch_terrier,0.064369,True,Old_English_sheepdog,0.043763,True
1912,853760880890318849,https://pbs.twimg.com/media/C9kq_bbVwAAuRZd.jpg,1,miniature_pinscher,0.292519,True,Chihuahua,0.120946,True,Rottweiler,0.119490,True
2004,877316821321428993,https://pbs.twimg.com/media/DCza_vtXkAQXGpC.jpg,1,Saluki,0.509967,True,Italian_greyhound,0.090497,True,golden_retriever,0.079406,True
1346,759197388317847553,https://pbs.twimg.com/media/Cok1_sjXgAU3xpp.jpg,1,kuvasz,0.511341,True,golden_retriever,0.076899,True,white_wolf,0.063269,False
797,691090071332753408,https://pbs.twimg.com/media/CZc-u7IXEAQHV1N.jpg,1,barrow,0.241637,False,tub,0.238450,False,bathtub,0.167285,False


In [13]:
df2.p1.value_counts(). head()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
Name: p1, dtype: int64

In [14]:
df2.p2.value_counts().head()

Labrador_retriever    104
golden_retriever       92
Cardigan               73
Chihuahua              44
Pomeranian             42
Name: p2, dtype: int64

In [15]:
df2.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [16]:
df2['jpg_url'].value_counts(). head()

https://pbs.twimg.com/media/CtVAvX-WIAAcGTf.jpg    2
https://pbs.twimg.com/media/Cwx99rpW8AMk_Ie.jpg    2
https://pbs.twimg.com/media/CwS4aqZXUAAe3IO.jpg    2
https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg    2
https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg    2
Name: jpg_url, dtype: int64

In [17]:
df2['tweet_id'].value_counts().head()

685532292383666176    1
826598365270007810    1
692158366030913536    1
714606013974974464    1
715696743237730304    1
Name: tweet_id, dtype: int64

In [18]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3715 entries, 0 to 3714
Data columns (total 11 columns):
id                3715 non-null int64
retweet_count     3715 non-null int64
favorite_count    3715 non-null int64
retweeted         3715 non-null bool
lang              3715 non-null object
created_at        3715 non-null object
contributors      0 non-null object
favorited         3715 non-null bool
coordinates       0 non-null object
place             1 non-null object
geo               0 non-null object
dtypes: bool(2), int64(3), object(6)
memory usage: 268.5+ KB


In [19]:
df3[df3.id.duplicated()].id.value_counts().head()

666337882303524864    4
666020888022790149    4
666421158376562688    4
666396247373291520    4
666353288456101888    4
Name: id, dtype: int64

In [20]:
df3.loc[df3["id"] == 666337882303524864 , ]

,id,retweet_count,favorite_count,retweeted,lang,created_at,contributors,favorited,coordinates,place,geo
22,666337882303524864,91,199,False,en,Mon Nov 16 19:31:45 +0000 2015,None,False,None,None,None
72,666337882303524864,91,199,False,en,Mon Nov 16 19:31:45 +0000 2015,None,False,None,None,None
122,666337882303524864,91,199,False,en,Mon Nov 16 19:31:45 +0000 2015,None,False,None,None,None
172,666337882303524864,91,199,False,en,Mon Nov 16 19:31:45 +0000 2015,None,False,None,None,None
3692,666337882303524864,91,199,False,en,Mon Nov 16 19:31:45 +0000 2015,None,False,None,None,None


In [21]:
df3.lang.value_counts()

en     3688
und       9
nl        6
in        5
et        2
ro        2
tl        1
eu        1
es        1
Name: lang, dtype: int64

In [22]:
df3.loc[df3["lang"] == "tl" , ]

,id,retweet_count,favorite_count,retweeted,lang,created_at,contributors,favorited,coordinates,place,geo
3548,668967877119254528,25,151,False,tl,Tue Nov 24 01:42:25 +0000 2015,None,False,None,None,None


In [23]:
df1.loc[df1["tweet_id"] == 668967877119254528 , ].text

2189    12/10 good shit Bubka\n@wane15
Name: text, dtype: object

In [24]:
df3.favorited.value_counts()

False    3715
Name: favorited, dtype: int64

In [25]:
df3.retweeted.value_counts()

False    3715
Name: retweeted, dtype: int64

In [26]:
df3.describe()

,id,retweet_count,favorite_count
count,3.715000e+03,3715.000000,3715.000000
mean,7.355399e+17,2722.948048,7765.391655
std,7.414148e+16,4645.440655,12323.785630
min,6.660209e+17,0.000000,0.000000
25%,6.733512e+17,456.500000,975.500000
50%,6.936227e+17,1138.000000,2871.000000
75%,8.022126e+17,3278.500000,9510.500000
max,8.924206e+17,76893.000000,142654.000000


In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [28]:
df1[df1.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [29]:
print(df1.source.value_counts().head())
print(df1.name.value_counts().head())
print(df1.doggo.value_counts().head())
print(df1.floofer.value_counts().head())
print(df1.pupper.value_counts().head())
print(df1.puppo.value_counts().head())

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64
None       745
a           55
Charlie     12
Oliver      11
Cooper      11
Name: name, dtype: int64
None     2259
doggo      97
Name: doggo, dtype: int64
None       2346
floofer      10
Name: floofer, dtype: int64
None      2099
pupper     257
Name: pupper, dtype: int64
None     2326
puppo      30
Name: puppo, dtype: int64


In [30]:
df1.name.sort_values().head()

1035     Abby
1021     Abby
938       Ace
1933     Acro
1327    Adele
Name: name, dtype: object

In [31]:
df1.loc[(df1['name'].str.islower()) & (df1['text'].str.contains('named'))].head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1853,675706639471788032,NaN,NaN,2015-12-12 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Sizzlin Menorah spaniel from Brookly...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675706639...,10,10,a,None,None,None,None
1955,673636718965334016,NaN,NaN,2015-12-06 22:54:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Lofted Aphrodisiac Terrier named Kip...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673636718...,10,10,a,None,None,None,None
2034,671743150407421952,NaN,NaN,2015-12-01 17:30:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Tuscaloosa Alcatraz named Jacob (Yac...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671743150...,11,10,a,None,None,None,None
2066,671147085991960577,NaN,NaN,2015-11-30 02:01:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Helvetica Listerine named Rufus. Thi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671147085...,9,10,a,None,None,None,None
2116,670427002554466305,NaN,NaN,2015-11-28 02:20:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Deciduous Trimester mix named Spork....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670427002...,9,10,a,None,None,None,None


In [32]:
df1[df1.text.str.contains('&amp;')].head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
262,842765311967449089,NaN,NaN,2017-03-17 15:51:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Indie. She's not a fan of baths but she's...,NaN,NaN,NaN,"https://www.gofundme.com/get-indie-home/,https...",12,10,Indie,None,None,None,None
273,840728873075638272,NaN,NaN,2017-03-12 00:59:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Pipsy. He is a fluffbal...,6.671522e+17,4.196984e+09,2015-11-19 01:27:25 +0000,https://twitter.com/dog_rates/status/667152164...,12,10,Pipsy,None,None,None,None
320,834458053273591808,NaN,NaN,2017-02-22 17:41:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Chester (bottom) &amp; Harold (top). They...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/834458053...,12,10,Chester,None,None,None,None
461,817536400337801217,NaN,NaN,2017-01-07 01:00:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Eugene &amp; Patti Melt. No matte...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/817536400...,12,10,Eugene,None,None,None,None
485,814578408554463233,NaN,NaN,2016-12-29 21:06:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Beau &amp; Wilbur. Wilbur ...,6.981954e+17,4.196984e+09,2016-02-12 17:22:12 +0000,https://twitter.com/dog_rates/status/698195409...,9,10,Beau,None,None,None,None


In [33]:
df1.rating_numerator.value_counts().head()

12    558
11    464
10    461
13    351
9     158
Name: rating_numerator, dtype: int64

In [34]:
df1.rating_denominator.value_counts().head()

10    2333
11       3
50       3
80       2
20       2
Name: rating_denominator, dtype: int64

In [35]:
df1[df1.rating_denominator != 10].head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,30582082.0,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None


## df1 : WeRateDogs Twitter Dataset 
> It includes 2356 entries and 17 columns. 

***Quality***
- Names column should be cleaned, there is unvalid records like a, the, an, the , very, unacceptable which is start with lowercase.
- timestamp,retweeted_status_timestamp column type should be date instead of object.
- text includes "'&amp;" instead of "&".
- invalid rating_denominator (different than 10). However, I checked them manually and they are true denominators, so there is no problematic extraction from text.
- Tweets_ids with no images however this problem will be solved when I joined with image prediction dataset. Because, I amnot expecting the prediction which dnot have any image.
- Missing values expressed as "none". (name, duppo, flopper, etc.)
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id data types should be intergs instead of float.
- We only want original ratings (no retweets) that have images
- From the source column, via which channel users connected to twitter. Therefore, column should be cleaned.
- excluding any tweet that is a retweet.

    
***Tidiness***
- joining with tables df2 and df3.
- getting together stages in one column.
- adding new features like gender, etc.


## df2 : Image Prediction Dataset
> Great dataset which has 2075 entries and 12 columns without any missing values. 

***Quality***
- Missed ID's exists in the dataset compare to d1
- Dublicated jpg_url
- p1, p2, p3 columns should be standardized as all lowercase and "-" expression should be removed.


***Tidiness***
- joining with tables df3 and df1.
- creating final dog prediction



## df3 : Tweepy API Dataset 
> It includes 3715 entries and 11 columns. 

***Quality***
- contributors, coordinates, place and geo features should be excluded due to high missing ratio.
- 1222  number of id variable is dublicated
- id=666337882303524864 exits 4 times in the dataset with same results.
- lang indicates that the language of tweet. I wondered how "tl" lang is texted. Then, I realized id=668967877119254528 is problematic input.


***Tidiness***
- joining with tables df2 and df1.
- favorited, retweeted columns includes always false inputs, therefore it should be excluded.

<a id='cleaning'></a>
## Cleaning Data

In this section, I am going to define the problem which are listed end of assesing data section, code the solution then test the code sequentialy.

In [56]:
df1_clean = df1.copy()
df2_clean = df2.copy()
df3_clean = df3.copy()

*** Define (1)***

Excluding any tweet that is a retweet from df1 dataset.

*** Code ***

In [57]:
df1_clean.drop(df1_clean[df1_clean['retweeted_status_id'].notnull()== True].index,inplace=True)

***Test***

In [58]:
df1_clean.retweeted_status_id.value_counts()

Series([], Name: retweeted_status_id, dtype: int64)

In [59]:
df1_clean.retweeted_status_id.sample(5)

1275   NaN
1177   NaN
2038   NaN
324    NaN
29     NaN
Name: retweeted_status_id, dtype: float64

*** Define (2)***

Cleaning all duplicated id's from df3 and duplicated urls from df2.

***Code***

In [60]:
df1_clean = df1_clean.drop_duplicates('tweet_id', keep = 'last')
df3_clean = df3_clean.drop_duplicates('id', keep = 'last')
df2_clean = df2_clean.drop_duplicates('jpg_url', keep = 'last')

***Test***

In [61]:
print(len(df1)) 
print(len(df1_clean)) 

2356
2175


In [62]:
print(len(df3)) 
print(len(df3_clean)) 

3715
2343


In [63]:
print(len(df2)) 
print(len(df2_clean)) 

2075
2009


*** Define (3)***

Joining all tables which are almost completed their important quality issues (duplication, retweets, etc.)

***Code***

In [64]:
df_all  = pd.merge(left=df1_clean, right=df2_clean, left_on='tweet_id', right_on='tweet_id', how='left')
df_all  = pd.merge(left=df3_clean, right=df_all, left_on='id', right_on='tweet_id', how='left')

***Test***

In [65]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2343 entries, 0 to 2342
Data columns (total 39 columns):
id                            2343 non-null int64
retweet_count                 2343 non-null int64
favorite_count                2343 non-null int64
retweeted                     2343 non-null bool
lang                          2343 non-null object
created_at                    2343 non-null object
contributors                  0 non-null object
favorited                     2343 non-null bool
coordinates                   0 non-null object
place                         1 non-null object
geo                           0 non-null object
tweet_id                      2174 non-null float64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2174 non-null object
source                        2174 non-null object
text                          2174 non-null object
retweeted_status_id           0 non-null float

*** Define (4)***

Drop uncessary columns and make unique 

***Code***

In [66]:
df_all  = df_all.drop(['id', 'retweeted_status_id', 'retweeted_status_user_id', 
                       'retweeted_status_timestamp','in_reply_to_status_id',
                       'in_reply_to_user_id','favorited', 'retweeted','contributors', 'coordinates',
                      'place', 'geo'], axis=1)

df_all = df_all.drop_duplicates('tweet_id', keep = 'last')

***Test***

In [67]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2342
Data columns (total 27 columns):
retweet_count         2175 non-null int64
favorite_count        2175 non-null int64
lang                  2175 non-null object
created_at            2175 non-null object
tweet_id              2174 non-null float64
timestamp             2174 non-null object
source                2174 non-null object
text                  2174 non-null object
expanded_urls         2116 non-null object
rating_numerator      2174 non-null float64
rating_denominator    2174 non-null float64
name                  2174 non-null object
doggo                 2174 non-null object
floofer               2174 non-null object
pupper                2174 non-null object
puppo                 2174 non-null object
jpg_url               1927 non-null object
img_num               1927 non-null float64
p1                    1927 non-null object
p1_conf               1927 non-null float64
p1_dog                1927 non

*** Define (5)***

Creating final prediction of image prediction

In [68]:
p_final = []
p_final_conf = []

def final_prediction(table) :
    if table['p1_dog'] == True:
        p_final.append(table['p1'])
        p_final_conf.append(table['p1_conf'])
    elif table['p2_dog'] == True:
        p_final.append(table['p2'])
        p_final_conf.append(table['p2_conf'])
    elif table['p3_dog'] == True:
        p_final.append(table['p3'])
        p_final_conf.append(table['p3_conf'])
    else:
        p_final.append('NaN')
        p_final_conf.append(0)

*** Code ***

In [69]:
df_all.apply(final_prediction, axis=1)
df_all['final_prediction'] = p_final
df_all['final_prediction_conf'] = p_final_conf

***Test***

In [70]:
df_all.final_prediction.value_counts().head()

NaN                   550
golden_retriever      143
Labrador_retriever    103
Pembroke               94
Chihuahua              87
Name: final_prediction, dtype: int64

In [71]:
df_all.final_prediction_conf.value_counts().head()

0.000000    550
0.873233      2
0.786089      2
0.240602      2
0.335692      1
Name: final_prediction_conf, dtype: int64

*** Define (6)***

From the source column, via which channel users connected to twitter. Therefore, column should be cleaned.

***Code***

In [72]:
df_all['source'] = df_all['source'].astype('category')

In [73]:
df_all['source'] = df_all['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

***Test***

In [74]:
df_all['source'].value_counts()

Twitter for iPhone     2041
Vine - Make a Scene      91
Twitter Web Client       31
TweetDeck                11
Name: source, dtype: int64

*** Define (7)***

Names column should be cleaned, there is unvalid records like a, the, an, the , very, unacceptable which is start with lowercase.

***Code***

In [75]:
df_all['text'] = df_all['text'].astype(object)

new_dog_names = []

for line in df_all.text:
    # Start with 'This is ' and the fisrt letter of the name is uppercase
    if str(line).startswith('This is ') and re.match(r'[A-Z].*', line.split()[2]):
        new_dog_names.append(line.split()[2].strip(',').strip('.'))
    # Start with 'Meet ' and the fisrt letter of the name is uppercase
    elif str(line).startswith('Meet ') and re.match(r'[A-Z].*', line.split()[1]):
        new_dog_names.append(line.split()[1].strip(',').strip('.'))
    # Start with 'Say hello to ' and the fisrt letter of the name is uppercase
    elif str(line).startswith('Say hello to ') and re.match(r'[A-Z].*', line.split()[3]):
        new_dog_names.append(line.split()[3].strip(',').strip('.'))
    # Start with 'Here we have ' and the fisrt letter of the name is uppercase
    elif str(line).startswith('Here we have ') and re.match(r'[A-Z].*', line.split()[3]):
        new_dog_names.append(line.split()[3].strip(',').strip('.'))
    # Contain 'named' and the fisrt letter of the name is uppercase
    elif 'named' in str(line) and re.match(r'[A-Z].*', line.split()[line.split().index('named') + 1]):
        new_dog_names.append(line.split()[line.split().index('named') + 1].strip(',').strip('.'))
    # No name specified or other style
    else:
        new_dog_names.append('NaN')
        
df_all['new_dog_names'] = new_dog_names

***Test***

In [76]:
df_all[['new_dog_names', 'name']].sample(10)

,new_dog_names,name
365,Beebop,Beebop
725,NaN,None
2032,Jax,Jax
2221,Skittles,Skittles
542,NaN,None
2143,Skye,Skye
271,Monty,Monty
1266,NaN,None
305,NaN,None
970,George,George


*** Define (8)***

Creating gender feature

***Code***

In [77]:
male = ['He', 'he', 'him', 'his', "he's", 'himself']
female = ['She', 'she', 'her', 'hers', 'herself', "she's"]

dog_gender = []

for text in df_all['text']:
    # Male
    if any(map(lambda v:v in male, str(text).split())):
        dog_gender.append('male')
    # Female
    elif any(map(lambda v:v in female, str(text).split())):
        dog_gender.append('female')
    # If group or not specified
    else:
        dog_gender.append('NaN')
        
df_all['dog_gender'] = dog_gender

***Test***

In [78]:
df_all.dog_gender.value_counts()

NaN       1275
male       667
female     233
Name: dog_gender, dtype: int64

*** Define (9)***

Converting null values to None correcting data types and removing null value on tweet_id coloumn.

***Code***

In [79]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2342
Data columns (total 31 columns):
retweet_count            2175 non-null int64
favorite_count           2175 non-null int64
lang                     2175 non-null object
created_at               2175 non-null object
tweet_id                 2174 non-null float64
timestamp                2174 non-null object
source                   2174 non-null category
text                     2174 non-null object
expanded_urls            2116 non-null object
rating_numerator         2174 non-null float64
rating_denominator       2174 non-null float64
name                     2174 non-null object
doggo                    2174 non-null object
floofer                  2174 non-null object
pupper                   2174 non-null object
puppo                    2174 non-null object
jpg_url                  1927 non-null object
img_num                  1927 non-null float64
p1                       1927 non-null object
p1_conf        

In [80]:
df_all = df_all.dropna(subset = ['tweet_id'])

In [81]:
df_all.loc[df_all['jpg_url'] == 'NaN', 'jpg_url'] = None
df_all.loc[df_all['expanded_urls'] == 'NaN', 'expanded_urls'] = None
df_all.loc[df_all['p1'] == 'NaN', 'p1'] = None
df_all.loc[df_all['p2'] == 'NaN', 'p2'] = None
df_all.loc[df_all['p3'] == 'NaN', 'p3'] = None
df_all.loc[df_all['final_prediction'] == 'NaN', 'final_prediction'] = None
df_all.loc[df_all['p1_dog'] == 'NaN', 'p1_dog'] = None
df_all.loc[df_all['p2_dog'] == 'NaN', 'p2_dog'] = None
df_all.loc[df_all['p3_dog'] == 'NaN', 'p3_dog'] = None


df_all['timestamp'] = pd.to_datetime(df_all.timestamp)
df_all['source'] = df_all['source'].astype('category')
df_all['rating_numerator'] = df_all['rating_numerator'].astype(float)
df_all['rating_denominator'] = df_all['rating_denominator'].astype(float)
df_all['dog_gender'] = df_all['dog_gender'].astype('category')

***Test***

In [82]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 0 to 2342
Data columns (total 31 columns):
retweet_count            2174 non-null int64
favorite_count           2174 non-null int64
lang                     2174 non-null object
created_at               2174 non-null object
tweet_id                 2174 non-null float64
timestamp                2174 non-null datetime64[ns]
source                   2174 non-null category
text                     2174 non-null object
expanded_urls            2116 non-null object
rating_numerator         2174 non-null float64
rating_denominator       2174 non-null float64
name                     2174 non-null object
doggo                    2174 non-null object
floofer                  2174 non-null object
pupper                   2174 non-null object
puppo                    2174 non-null object
jpg_url                  1927 non-null object
img_num                  1927 non-null float64
p1                       1927 non-null object
p1_conf

*** Define (10)***

Creating date and time from timestamp

***Code***

In [83]:
df_all['date'] = df_all['timestamp'].apply(lambda time: time.strftime('%m-%d-%Y'))
df_all['time'] = df_all['timestamp'].apply(lambda time: time.strftime('%H:%M'))

***Test***

In [84]:
df_all[['time', 'date']].sample(10)

,time,date
1075,02:00,06-01-2016
8,16:25,07-27-2017
1381,17:01,02-17-2016
2238,20:47,11-20-2015
2209,16:31,11-22-2015
722,16:03,09-29-2016
2134,05:42,11-26-2015
1418,20:23,02-10-2016
2048,03:18,11-30-2015
568,17:23,11-21-2016


*** Define (11)***

Creating a stage coloumn combining dog stages features.

*** Code ***

In [85]:
df_all['stage'] = df_all[['doggo', 'floofer','pupper','puppo']].apply(lambda x: ''.join(x), axis=1)

df_all['stage'].replace("NoneNoneNoneNone","None ", inplace=True)
df_all['stage'].replace("doggoNoneNoneNone","doggo", inplace=True)
df_all['stage'].replace("NoneflooferNoneNone","floofer", inplace=True)
df_all['stage'].replace("NoneNonepupperNone","pupper", inplace=True)
df_all['stage'].replace("NoneNoneNonepuppo","puppo", inplace=True)

*** Test ***

In [86]:
df_all['stage'].value_counts()

None                    1830
pupper                   224
doggo                     75
puppo                     24
doggoNonepupperNone       10
floofer                    9
doggoNoneNonepuppo         1
doggoflooferNoneNone       1
Name: stage, dtype: int64

*** Define (12)***

Replacing underscore with space.

*** Code ***

In [87]:
df_all['p1'] = df_all['p1'].str.replace('_', ' ')
df_all['p1'] = df_all['p1'].map(lambda x: x if type(x)!=str else x.lower())
df_all['p2'] = df_all['p2'].str.replace('_', ' ')
df_all['p2'] = df_all['p2'].map(lambda x: x if type(x)!=str else x.lower())
df_all['p3'] = df_all['p3'].str.replace('_', ' ')
df_all['p3'] = df_all['p3'].map(lambda x: x if type(x)!=str else x.lower())
df_all['final_prediction'] = df_all['final_prediction'].str.replace('_', ' ')
df_all['final_prediction'] = df_all['final_prediction'].map(lambda x: x if type(x)!=str else x.lower())

*** Test ***

In [88]:
df_all[['p1','p2','p3','final_prediction']].sample(10)

,p1,p2,p3,final_prediction
846,NaN,NaN,NaN,None
1874,miniature poodle,toy poodle,maltese dog,miniature poodle
720,pembroke,golden retriever,collie,pembroke
890,labrador retriever,chihuahua,american staffordshire terrier,labrador retriever
717,briard,soft-coated wheaten terrier,lhasa,briard
842,NaN,NaN,NaN,None
689,NaN,NaN,NaN,None
1413,washbasin,paper towel,toilet tissue,None
1197,golden retriever,labrador retriever,chow,golden retriever
1488,toy poodle,teddy,miniature poodle,toy poodle


In [89]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 0 to 2342
Data columns (total 34 columns):
retweet_count            2174 non-null int64
favorite_count           2174 non-null int64
lang                     2174 non-null object
created_at               2174 non-null object
tweet_id                 2174 non-null float64
timestamp                2174 non-null datetime64[ns]
source                   2174 non-null category
text                     2174 non-null object
expanded_urls            2116 non-null object
rating_numerator         2174 non-null float64
rating_denominator       2174 non-null float64
name                     2174 non-null object
doggo                    2174 non-null object
floofer                  2174 non-null object
pupper                   2174 non-null object
puppo                    2174 non-null object
jpg_url                  1927 non-null object
img_num                  1927 non-null float64
p1                       1927 non-null object
p1_conf

*** Define (13)***

Excluding the null values from final_prediction dataset because I will use final_prediction feature as predictive variable.

***Code***

In [90]:
df_all[["p1","p1_conf", "p1_dog","p2","p2_conf", "p2_dog","p3","p3_conf",  "p3_dog","final_prediction", "final_prediction_conf"]].sample(10)

,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,final_prediction,final_prediction_conf
1253,chihuahua,0.945629,True,pomeranian,0.019204,True,west highland white terrier,0.010134,True,chihuahua,0.945629
114,greater swiss mountain dog,0.622752,True,appenzeller,0.158463,True,entlebucher,0.148115,True,greater swiss mountain dog,0.622752
2248,electric fan,0.984377,False,spotlight,0.007737,False,lampshade,0.001901,False,None,0.000000
1659,curly-coated retriever,0.482288,True,flat-coated retriever,0.315286,True,great dane,0.062179,True,curly-coated retriever,0.482288
1465,dining table,0.383448,False,grey fox,0.103191,False,siamese cat,0.098256,False,None,0.000000
677,beach wagon,0.362925,False,minivan,0.304759,False,limousine,0.101702,False,None,0.000000
1089,studio couch,0.944692,False,four-poster,0.007942,False,quilt,0.006302,False,None,0.000000
1470,golden retriever,0.989333,True,labrador retriever,0.007946,True,kuvasz,0.000749,True,golden retriever,0.989333
547,labrador retriever,0.372776,True,golden retriever,0.343666,True,great pyrenees,0.067242,True,labrador retriever,0.372776
1293,golden retriever,0.637225,True,bloodhound,0.094542,True,cocker spaniel,0.069797,True,golden retriever,0.637225


In [91]:
df_all = df_all.dropna(subset = ['final_prediction'])

*** Test***

In [92]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 1 to 2342
Data columns (total 34 columns):
retweet_count            1625 non-null int64
favorite_count           1625 non-null int64
lang                     1625 non-null object
created_at               1625 non-null object
tweet_id                 1625 non-null float64
timestamp                1625 non-null datetime64[ns]
source                   1625 non-null category
text                     1625 non-null object
expanded_urls            1625 non-null object
rating_numerator         1625 non-null float64
rating_denominator       1625 non-null float64
name                     1625 non-null object
doggo                    1625 non-null object
floofer                  1625 non-null object
pupper                   1625 non-null object
puppo                    1625 non-null object
jpg_url                  1625 non-null object
img_num                  1625 non-null float64
p1                       1625 non-null object
p1_conf

*** Define (14)***

Cleaning rating numerators

*** Code ***

In [93]:
ratings = df_all.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True)
df_all['new_numerator'] = ratings[0]
df_all['new_denominator'] = ratings[1]

*** Test ***

In [94]:
df_all[df_all['tweet_id'] == 786709082849828864]

,retweet_count,favorite_count,lang,created_at,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,...,p3_dog,final_prediction,final_prediction_conf,new_dog_names,dog_gender,date,time,stage,new_numerator,new_denominator
685,6814,19796,en,Thu Oct 13 23:23:56 +0000 2016,7.867091e+17,2016-10-13 23:23:56,Twitter for iPhone,"This is Logan, the Chow who lived. He solemnly...",https://twitter.com/dog_rates/status/786709082...,75.0,...,True,pomeranian,0.467321,Logan,male,10-13-2016,23:23,None,9.75,10


<a id='storing'></a>
## Storing/Exporting Data

In this section, I am going to save the cleaned for further both exploratory and predictive data analysis.

In [95]:
df_all.to_csv('twitter_archive_master_v2.csv', index=False, encoding = 'utf-8')

In [96]:
df_all_test = pd.read_csv('twitter_archive_master_v2.csv')
df_all_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625 entries, 0 to 1624
Data columns (total 36 columns):
retweet_count            1625 non-null int64
favorite_count           1625 non-null int64
lang                     1625 non-null object
created_at               1625 non-null object
tweet_id                 1625 non-null float64
timestamp                1625 non-null object
source                   1625 non-null object
text                     1625 non-null object
expanded_urls            1625 non-null object
rating_numerator         1625 non-null float64
rating_denominator       1625 non-null float64
name                     1625 non-null object
doggo                    1625 non-null object
floofer                  1625 non-null object
pupper                   1625 non-null object
puppo                    1625 non-null object
jpg_url                  1625 non-null object
img_num                  1625 non-null float64
p1                       1625 non-null object
p1_conf          